# DF

In [ ]:
import marimo as mo
import polars as pl

from r_translation import join

matching, contentious = join()
matching = matching.with_columns(
    genus=pl.when(
        (pl.col("genus").is_null()) & (pl.col("specificEpithet").is_not_null())
    )
    .then(pl.col("taxonName").str.split(" ").list[0])
    .otherwise("genus")
)

bos_df columns: ['speciesId', 'phylum', 'class', 'subclass', 'order', 'sub-order', 'infraorder', 'section', 'subsection', 'superfamily', 'family', 'subfamily', 'tribe', 'genus', 'subgenus', 'specificEpithet', 'infraspecificEpithet', 'taxonName', 'source_file', 'taxon', 'superorder', 'domain', 'kingdom', 'section/parvorder', 'Column2', 'infraclass', 'commonName', 'species2', 'clade 1', 'clade 2', 'clade 3', 'genericName']


taxon.csv columns, odict_keys(['taxonID', 'genericName', 'genus', 'specificEpithet', 'infraspecificEpithet', 'taxonomicStatus', 'acceptedNameUsageID'])


In [ ]:
contentious.collect_schema().keys()

odict_keys(['speciesId', 'matched_taxonID', 'acceptedNameUsageID', 'taxonName', 'domain', 'kingdom', 'phylum', 'class', 'subclass', 'superorder', 'order', 'sub-order', 'infraorder', 'section', 'subsection', 'superfamily', 'family', 'subfamily', 'tribe', 'genus', 'subgenus', 'genericName', 'specificEpithet', 'infraspecificEpithet'])

nomatch lazyframe is for BOS data point that has no current match in gbif data set.

In [ ]:
nomatch = (
    matching.filter(pl.col("matched_taxonID").is_null())
    .with_columns(
        taxonRank=pl.lit("BOSuncornirmedSpecies"),
        taxonomicStatus=pl.lit("BOSunformired"),
        parentNameUsageID=pl.lit(None),
    )
    .fill_null("")
    .collect()
)
nomatch

speciesId,matched_taxonID,acceptedNameUsageID,taxonName,domain,kingdom,phylum,class,subclass,superorder,order,sub-order,infraorder,section,subsection,superfamily,family,subfamily,tribe,genus,subgenus,genericName,specificEpithet,infraspecificEpithet,taxonRank,taxonomicStatus,parentNameUsageID
str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,null
"""A-Anne-Clit-Hirudinea-000001""",null,null,"""Glossiphoniidae sp. ""1""""","""Eukarya""","""Animalia""","""Annelida""","""Clitellata""","""Hirudinea""","""""","""Rhynchobdellida""","""""","""""","""""","""""","""""","""Glossiphoniidae""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000002""",null,null,"""Glossiphoniidae sp. ""2""""","""Eukarya""","""Animalia""","""Annelida""","""Clitellata""","""Hirudinea""","""""","""Rhynchobdellida""","""""","""""","""""","""""","""""","""Glossiphoniidae""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000003""",null,null,"""Glossiphoniidae sp. ""3""""","""Eukarya""","""Animalia""","""Annelida""","""Clitellata""","""Hirudinea""","""""","""Rhynchobdellida""","""""","""""","""""","""""","""""","""Glossiphoniidae""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000004""",null,null,"""Hirudinidae sp. ""1""""","""Eukarya""","""Animalia""","""Annelida""","""Clitellata""","""Hirudinea""","""""","""Arhynchobdellida""","""""","""""","""""","""""","""""","""Hirudinidae""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000005""",null,null,"""Hirudinidae sp. ""2""""","""Eukarya""","""Animalia""","""Annelida""","""Clitellata""","""Hirudinea""","""""","""Arhynchobdellida""","""""","""""","""""","""""","""""","""Hirudinidae""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""A-Hemi-Enteropneusta-000010""",null,null,"""Acorn worm sp.""","""Eukarya""","""Animalia""","""Hemichordata""","""Enteropneusta""","""""","""""","""blank-UnID""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Hemi-Enteropneusta-000011""",null,null,"""Acorn worm sp.""","""Eukarya""","""Animalia""","""Hemichordata""","""Enteropneusta""","""""","""""","""blank-UnID""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Phoronida-000002""",null,null,"""Phoronida sp.""","""Eukarya""","""Animalia""","""Phoronida""","""blank-Phor""","""""","""""","""blank-Phor""","""""","""""","""""","""""","""""","""Phoronidae""","""""","""""","""""","""""","""""","""""","""""","""BOSuncornirmedSpecies""","""BOSunformired""",null


In [ ]:
matching_with_populated_match_taxonID = matching.filter(
    pl.col("matched_taxonID").is_not_null()
).with_columns(
    taxonRank=pl.lit(None),
    taxonomicStatus=pl.lit(None),
    parentNameUsageID=pl.lit(None),
)

# repeated_accepted_taxons = RAT

In [ ]:
repeated_accepted_taxons = (
    pl.scan_csv("gbif/Taxon.tsv", separator="\t", quote_char=None, cache=True)
    .filter(pl.col("taxonomicStatus") == pl.lit("accepted"))
    .filter(pl.col("kingdom").is_in(["Animalia", "Plantae"]))
    .filter(~pl.col("canonicalName").is_null())
    .filter(pl.col("canonicalName") != "")
    .sort("canonicalName")
    .filter(pl.col("canonicalName").is_duplicated())
)

In [ ]:
priority_columns = [
    "infraspecificEpithet",
    "specificEpithet",
    "genus",
    "family",
    "order",
    "class",
    "phylum",
    "kingdom",
]

_schema = {
    "feature_that_is_equal_to_canonicalName": pl.String,
    "matches": pl.String,
}
_r = (repeated_accepted_taxons).collect()
RAT_interim = pl.DataFrame(schema=_schema)
for _col in priority_columns:
    _canonical_names = (
        (
            _r.filter(pl.col("canonicalName") == pl.col(_col))
            .select("canonicalName")
            .unique()
        )
        .to_series()
        .to_list()
    )
    if len(_canonical_names) != 0:
        _row = pl.DataFrame(
            data={
                "feature_that_is_equal_to_canonicalName": _col,
                "matches": _canonical_names,
            },
            schema=_schema,
        )
        RAT_interim = RAT_interim.vstack(_row)
RAT_feats = (
    RAT_interim.group_by("matches")
    .agg(pl.col("feature_that_is_equal_to_canonicalName").str.join(", "))
    .with_columns(
        pl.col("feature_that_is_equal_to_canonicalName").str.split(", ")
    )
    .sort("feature_that_is_equal_to_canonicalName")
)

# matching attempt

In [ ]:
RAT_feats

matches,feature_that_is_equal_to_canonicalName
str,list[str]
"""Micrognathozoa""","[""class"", ""phylum""]"
"""Sirenia""","[""family"", ""order""]"
"""Passeriformes""","[""family"", ""order""]"
"""Psychopsis""","[""genus""]"
"""Parlatoria""","[""genus""]"
…,…
"""Diplura""","[""genus"", ""order"", ""class""]"
"""Arthropoda""","[""genus"", ""phylum""]"
"""Ctenophora""","[""genus"", ""phylum""]"


In [ ]:
updated_to_matching = []
all_taxon_data_to_be_selected_from = []
still_no_match = []
_collected_repeated_taxons = repeated_accepted_taxons.collect().fill_null("")
_reversed_priority_columns = priority_columns.copy()
_reversed_priority_columns.reverse()
for _col in _reversed_priority_columns:
    for _match in RAT_feats["matches"]:
        # skipping those that are not in RAT_feats
        if _match not in nomatch[_col].to_list():
            continue

        print("-------\nFeature:", _col, ",Name:", _match)

        taxon_data_to_select_from = _collected_repeated_taxons.filter(
            pl.col("canonicalName") == _match
        ).select(["taxonID"] + _reversed_priority_columns)

        # Select from the two rows what taxon data to select.
        selected_row_int = -1
        for i, _tuple in enumerate(
            taxon_data_to_select_from.select(
                ["taxonID"] + priority_columns
            ).iter_rows()
        ):
            col_index = priority_columns.index(_col)
            _x = _tuple[col_index]
            if not bool(_x):
                selected_row_int = i  # Since there are usually only two rows from taxon_data_to_select_from, the results is either 0 or 1
                break
        assert selected_row_int != -1

        # has_predicament1 is when kingdom to genus all has values and it is kind of leveled between the two rows of the taxon data.
        has_predicament1 = False
        _l = taxon_data_to_select_from["family"].to_list()
        if _col == "genus" and _l[0] != None and _l[1] != None:
            print("same level detected")
            has_predicament1 = True

        chosen_taxonId = taxon_data_to_select_from[selected_row_int, 0]
        other_taxonId = taxon_data_to_select_from[
            int(not bool(selected_row_int)), 0
        ]

        # Getting the _no_match_subset_to_update section
        _no_match_subset_to_update = nomatch.filter(
            pl.col(_col) == _match,
        )
        _x = _reversed_priority_columns[:-3]
        feature_to_find_nulls = (
            _x[_reversed_priority_columns.index(_col) + 1]
            if len(_x) > _reversed_priority_columns.index(_col) + 1
            else None
        )  # just the next feature to find null
        if feature_to_find_nulls is not None:
            print("feature_to_find_nulls", feature_to_find_nulls)
            _no_match_subset_to_update = _no_match_subset_to_update.filter(
                pl.col(feature_to_find_nulls) == ""
            )
            if _no_match_subset_to_update.is_empty():
                still_no_match_subset = nomatch.filter(
                    pl.col(_col) == _match,
                ).with_columns(
                    current_feature=pl.lit(_col), current_name=pl.lit(_match)
                )
                still_no_match.append(still_no_match_subset)
                print(
                    "\n\n\n\n\nskipping",
                    _col,
                    _match,
                    "\n this _no_match_subset_to_update dataframe doesn't have the same tax rank level\n",
                    still_no_match_subset.select(_reversed_priority_columns),
                )
                print("---------")
                continue

        # Settling predicament one
        # this predicament thing should only happen when the subset is of only 1 row.
        if has_predicament1 and _no_match_subset_to_update.shape[0] == 1:
            match = True
            for _col1 in ["class", "order", "family"]:
                match *= (
                    _no_match_subset_to_update[_col1].item()
                    == taxon_data_to_select_from[selected_row_int, :][
                        _col1
                    ].item()
                )
            if not match:
                match = True
                for _col1 in ["class", "order", "family"]:
                    match *= (
                        _no_match_subset_to_update[_col1].item()
                        == taxon_data_to_select_from[
                            int(not bool(selected_row_int)), :
                        ][_col1].item()
                    )  # searching in the three columns of interests for matches
                assert match == True  # Double check that there is no unmatch.
                print("changing chosen taxonId")
                temp = chosen_taxonId
                chosen_taxonId = other_taxonId
                other_taxonId = temp

        # turning these rows to matching by filling parentNameUsageID
        _no_match_subset_to_update = _no_match_subset_to_update.with_columns(
            parentNameUsageID=pl.when(
                (pl.col("infraspecificEpithet").is_null())
                & (pl.col("specificEpithet").is_not_null())
                & (pl.col("genus").is_not_null())
            )
            .then(pl.lit(other_taxonId))
            .otherwise(pl.lit(chosen_taxonId))
        )

        # Debugging for matching data from gbif to new matched rows
        print(
            "taxon_data_to_select_from\n",
            taxon_data_to_select_from,
        )
        print("parentNameUsageID chosen:", chosen_taxonId)
        print("no_match_subset_to_update ")
        print(_no_match_subset_to_update.select(_reversed_priority_columns))
        print("---------")

        updated_to_matching.append(_no_match_subset_to_update)
        # all_taxon_data_to_be_selected_from.append(taxon_data_to_select_from)

# turn them into dataframes and writing to csv files
updated_to_matching = pl.concat(
    updated_to_matching, rechunk=True, parallel=True
)
updated_to_matching.write_csv("updated_to_matching.csv")
updated_to_matching.write_csv("first_matches_set_from_wrangling.csv")
# all_taxon_data_to_be_selected_from = pl.concat(
#     all_taxon_data_to_be_selected_from, rechunk=True, parallel=True
# )
# all_taxon_data_to_be_selected_from.write_csv(
#     "all_taxon_data_to_be_selected_from.csv"
# )
still_no_match = pl.concat(still_no_match, rechunk=True, parallel=True)
# still_no_match.write_csv("still_no_match.csv")

## still no match stuff

### `current_feature` uniques

In [ ]:
still_no_match["current_feature"].unique()

current_feature
str
"""order"""
"""phylum"""


### `current_name` uniques

In [ ]:
still_no_match["current_name"].unique()

current_name
str
"""Arthropoda"""
"""Siluriformes"""
"""Passeriformes"""
"""Ctenophora"""


## number of rows in still no match

In [ ]:
still_no_match.shape[0]

2287

# second matching wrangle of new `Taxons` df with `still_no_match`

In [ ]:
taxons = (
    pl.scan_csv("gbif/Taxon.tsv", separator="\t", quote_char=None, cache=True)
    .filter(
        pl.col("taxonomicStatus") == pl.lit("accepted"),
        pl.col("taxonRank") != "unranked",
    )
    .filter(pl.col("kingdom").is_in(["Animalia", "Plantae"]))
    .select(["taxonID"] + priority_columns)
)
taxons.collect().shape

## stop at class, there is only current_feature = phylum matches

In [ ]:
priority_columns

In [ ]:
## stop at class, there is only current_feature = phylum matches
_filter = (
    pl.col("class") != "",
    pl.col("order") == "",
    pl.col("family") == "",
    pl.col("genus") == "",
    pl.col("specificEpithet") == "",
    pl.col("infraspecificEpithet") == "",
    pl.col("current_feature") == "phylum",
)
match_on_class = still_no_match.with_columns(
    name_to_match=pl.when(_filter).then("class").otherwise(None)
).filter(_filter)

_x1 = (
    still_no_match.filter(_filter)
    .select("class", "phylum")
    .group_by("class", "phylum")
    .len()
)

_taxons_subset = taxons.filter(
    pl.col("family").is_null(),
    pl.col("order").is_null(),
    pl.col("genus").is_null(),
    pl.col("class").is_in(_x1["class"].unique()),
    pl.col("specificEpithet").is_null(),
    pl.col("infraspecificEpithet").is_null(),
    pl.col("phylum").is_in(_x1["phylum"].unique()),
).select("taxonID", "class")
match_on_class = (
    pl.LazyFrame(match_on_class)
    .join(
        _taxons_subset,
        on="class",
    )
    .with_columns(
        parentNameUsageID=pl.when(pl.col("taxonID").is_not_null())
        .then("taxonID")
        .otherwise("parentNameUsageID")
    )
    .drop("taxonID")
).collect()

match_on_class.shape

/tmp/marimo_230255/__marimo__cell_xXTn_.py:47: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  ).collect()


In [ ]:
match_on_class.group_by("parentNameUsageID", "name_to_match").len()

parentNameUsageID,name_to_match,len
i64,str,u32
353,"""Ostracoda""",16
10713444,"""Collembola""",2
365,"""Tentaculata""",7
11545536,"""Copepoda""",17


In [ ]:
match_on_class.shape

## stops at order there is only current_feature = phylum matches

In [ ]:
## stops at order there is only current_feature = phylum matches
_filter = (
    pl.col("class") != "",
    pl.col("order") != "",
    pl.col("family") == "",
    pl.col("genus") == "",
    pl.col("specificEpithet") == "",
    pl.col("infraspecificEpithet") == "",
    pl.col("current_feature") == "phylum",
)
match_on_order = still_no_match.with_columns(
    name_to_match=pl.when(_filter).then("order").otherwise(None)
).filter(_filter)
_x1 = (
    still_no_match.filter(_filter)
    .select("order", "phylum")
    .group_by("order", "phylum")
    .len()
)

_taxons_subset = (
    taxons.filter(
        pl.col("family").is_null(),
        pl.col("order").is_in(_x1["order"].unique().to_list()),
        pl.col("genus").is_null(),
        pl.col("specificEpithet").is_null(),
        pl.col("infraspecificEpithet").is_null(),
        pl.col("phylum").is_in(_x1["phylum"].unique().to_list()),
    )
    .select("taxonID", "order")
    .collect()
)
match_on_order = (
    match_on_order.join(
        _taxons_subset,
        on="order",
    )
    .with_columns(
        parentNameUsageID=pl.when(pl.col("taxonID").is_not_null())
        .then("taxonID")
        .otherwise("parentNameUsageID")
    )
    .drop("taxonID")
)
match_on_order.shape

In [ ]:
match_on_order.group_by("parentNameUsageID", "name_to_match").len()

parentNameUsageID,name_to_match,len
i64,str,u32
788,"""Mantodea""",2
643,"""Isopoda""",54
1500,"""Cumacea""",6
637,"""Decapoda""",103
787,"""Plecoptera""",1
…,…,…
1496,"""Araneae""",1
1373,"""Stomatopoda""",10
841,"""Mysida""",7


In [ ]:
match_on_order.shape

## stops at family, current_feature contains both phylum and order

In [ ]:
## stops at family there is only current_feature = phylum matches
_filter = (
    pl.col("class") != "",
    pl.col("order") != "",
    pl.col("family") != "",
    pl.col("genus") == "",
    pl.col("specificEpithet") == "",
    pl.col("infraspecificEpithet") == "",
)
match_on_family = still_no_match.with_columns(
    name_to_match=pl.when(_filter).then("family").otherwise(None)
).filter(_filter)
_x1 = (
    still_no_match.filter(_filter)
    .select("family", "phylum", "order")
    .group_by("family", "phylum", "order")
    .len()
)

_taxons_subset = (
    taxons.filter(
        pl.col("family").is_in(_x1["family"].unique().to_list()),
        pl.col("genus").is_null(),
        pl.col("specificEpithet").is_null(),
        pl.col("infraspecificEpithet").is_null(),
        pl.col("phylum").is_in(_x1["phylum"].unique().to_list()),
        pl.col("order").is_in(_x1["order"].unique().to_list()),
    )
    .select("taxonID", "family")
    .collect()
)
match_on_family = (
    match_on_family.join(
        _taxons_subset,
        on="family",
    )
    .with_columns(
        parentNameUsageID=pl.when(pl.col("taxonID").is_not_null())
        .then("taxonID")
        .otherwise("parentNameUsageID")
    )
    .drop("taxonID")
)
match_on_family.shape

In [ ]:
match_on_family.group_by(
    "parentNameUsageID", "name_to_match", "current_feature"
).len().filter(pl.col("current_feature") == "phylum")

parentNameUsageID,name_to_match,current_feature,len
i64,str,str,u32
9644,"""Eucnemidae""","""phylum""",1
5925,"""Gryllidae""","""phylum""",28
8888,"""Idoteidae""","""phylum""",1
2783,"""Menoponidae""","""phylum""",18
6981,"""Erotylidae""","""phylum""",1
…,…,…,…
7780,"""Chrysomelidae""","""phylum""",11
7366,"""Desidae""","""phylum""",1
8672,"""Pinnotheridae""","""phylum""",3


In [ ]:
match_on_family.group_by(
    "parentNameUsageID", "name_to_match", "current_feature"
).len().filter(pl.col("current_feature") == "order")

parentNameUsageID,name_to_match,current_feature,len
i64,str,str,u32
5164,"""Plotosidae""","""order""",4
4512,"""Ariidae""","""order""",3


In [ ]:
match_on_family.shape

## genus

There is nothing for rank genus.

In [ ]:
_filter = (
    pl.col("class") != "",
    pl.col("order") != "",
    pl.col("family") != "",
    pl.col("genus") != "",
    pl.col("specificEpithet") == "",
    pl.col("infraspecificEpithet") == "",
)
_g = still_no_match.filter(_filter)
_g

speciesId,matched_taxonID,acceptedNameUsageID,taxonName,domain,kingdom,phylum,class,subclass,superorder,order,sub-order,infraorder,section,subsection,superfamily,family,subfamily,tribe,genus,subgenus,genericName,specificEpithet,infraspecificEpithet,taxonRank,taxonomicStatus,parentNameUsageID,current_feature,current_name
str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,null,str,str


## stops at specificEpithet there is only current_feature = phylum matches

### something is wrong with some specificEpithet and infraSpecificEpithet for both pl.col('current_feature').is_in(['phylum','order']) cant be found in gbif.

In [ ]:
## stops at family there is only current_feature = phylum matches
_filter = (
    pl.col("class") != "",
    pl.col("order") != "",
    pl.col("family") != "",
    pl.col("genus") != "",
    pl.col("specificEpithet") != "",
    pl.col("infraspecificEpithet") == "",
)

to_be_match_on_specificEpithet = still_no_match.filter(_filter)

_x1 = (
    still_no_match.filter(_filter)
    .select(
        "specificEpithet",
        "genus",
    )
    .group_by(
        "specificEpithet",
        "genus",
    )
    .len()
)

_taxons_subset = (
    taxons.filter(
        pl.col("specificEpithet").is_in(
            _x1["specificEpithet"].unique().to_list()
        ),
        pl.col("genus").is_in(_x1["genus"].unique().to_list()),
    )
    .select(
        "taxonID",
        "specificEpithet",
        "genus",
    )
    .collect()
)

# TAXON.TSV doesn't have pl.col("genus")=='Dicaeum',pl.col('specificEpithet')=='chysorrheum'
match_on_specificEpithet = (
    to_be_match_on_specificEpithet.join(
        _taxons_subset,
        on=["genus", "specificEpithet"],
        # how="left"
    )
    .with_columns(
        parentNameUsageID=pl.when(pl.col("taxonID").is_not_null())
        .then("taxonID")
        .otherwise("parentNameUsageID")
    )
    .drop("taxonID")
)
match_on_specificEpithet.shape
# match_on_specificEpithet.write_csv("matching_attempt_on_specificEpithet.csv")

## infraspecificepithet

different filter because some don't have specificEpithet either, if I am not wrong

In [ ]:
# InfraspecificEpithet

## stops at family there is only current_feature = phylum matches
_filter = (
    pl.col("class") != "",
    pl.col("order") != "",
    pl.col("family") != "",
    pl.col("genus") != "",
    pl.col("infraspecificEpithet") != "",
)  # different filter because some don't have specificEpithet either, if I am not wrong
to_be_match_on_infraspecificEpithet = still_no_match.filter(_filter)

_x1 = (
    still_no_match.filter(_filter)
    .select(
        "infraspecificEpithet",
        "genus",
    )
    .group_by(
        "infraspecificEpithet",
        "genus",
    )
    .len()
)

_taxons_subset = (
    taxons.filter(
        pl.col("infraspecificEpithet").is_in(
            _x1["infraspecificEpithet"].unique().to_list()
        ),
        pl.col("genus").is_in(_x1["genus"].unique().to_list()),
    )
    .select(
        "taxonID",
        "infraspecificEpithet",
        "genus",
    )
    .collect()
)


# TAXON.TSV doesn't have pl.col("genus")=='Dicaeum',pl.col('specificEpithet')=='chysorrheum'
match_on_infraspecificEpithet = (
    to_be_match_on_infraspecificEpithet.join(
        _taxons_subset,
        on=["genus", "infraspecificEpithet"],
    )
    .with_columns(
        parentNameUsageID=pl.when(pl.col("taxonID").is_not_null())
        .then("taxonID")
        .otherwise("parentNameUsageID")
    )
    .drop("taxonID")
)
# print(match_on_infraspecificEpithet.select(priority_columns))

#combining updated parent id rows

todo: specificEpithet infraSpecificepithet

In [ ]:
match_on_class.columns

unfortunately my 2nd wrangle did some redundant work. I don't know how to remove this wrangling

In [ ]:
_to_drop = ["current_feature", "current_name", "name_to_match"]
_to_drop2 = ["current_feature", "current_name"]
_new_match = pl.concat(
    [
        match_on_class.drop(_to_drop),
        match_on_order.drop(_to_drop),
        match_on_family.drop(_to_drop),
        match_on_specificEpithet.drop(_to_drop2),
        match_on_infraspecificEpithet.drop(_to_drop2),
    ]
)
_redundant = updated_to_matching.with_columns().join(
    _new_match, on="speciesId"
)  # removing redundant
print(
    "parentNameUsageID on the redundant check:",
    (
        _redundant["parentNameUsageID"]
        != _redundant["parentNameUsageID_right"]
    ).any(),
)
_new_match.write_csv("second_matches_set_from_wrangling.csv")

_new_match = _new_match.join(
    _redundant.select("speciesId"), on="speciesId", how="anti"
)
print("new match shape", _new_match.shape)
print()
_new_match = pl.concat(
    [
        _new_match,
        updated_to_matching.with_columns(
            pl.col("parentNameUsageID").cast(pl.Int64)
        ),
    ]
)
_new_match.write_csv("updated_to_matching.csv")
print("shape of all matches:", _new_match.shape[0])
# print("new match unique speciesId",_new_match.select("speciesId").unique())
print("shape of nomatch before any matches", nomatch.shape[0])
_nomatch = nomatch.join(_new_match.select("speciesId"), on="speciesId", how="anti")
print(_x.shape)

parentNameUsageID on the redundant check: False
new match shape (2035, 27)



shape of all matches: 2502
shape of nomatch before any matches 8445
(5943, 27)


In [ ]:
5943 + 2502

8445